In [19]:
%%capture
!"C:\Program Files\Python310\python.exe" -m pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 torch
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" "C:\Program Files\Python310\python.exe" -m pip install llama-cpp-python

## LLM und Tokenizer laden

In [22]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


False


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    #"./models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85",
    device_map="cuda", 
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")#"./models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85",)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False,
)

`torch_dtype` is deprecated! Use `dtype` instead!


AttributeError: module 'torch.cuda' has no attribute 'is_floating_point'

# Übung
## Aufgabe 1:
Erzeugen Sie einen one_shot_prompt bei dem Sie das Wort "Gedankenexperiment" als user und assistent erklären. Anschließend machen Sie ein Beispiel zum  Wort "Brezn" und lassen Sie sich darauf hin ein Satz mit Brezn bilden.

In [7]:
one_shot_prompt = [
{"role":"user", "content": "Ein Gedankenexperiment ist wenn man etwas in Gedanken durchspielt"},
{"role": "assistent", "content": "Ich mache ein Gedankenexperiment, wieviel kilonewton der Knoten einer Brezn halten kann."},
{"role":"user", "content": "Ein Beispiel für ein Gedankenexperiment mit Brezn ist:"},
]
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|user|>
Ein Gedankenexperiment ist wenn man etwas in Gedanken durchspielt<|end|>
<|user|>
Ein Beispiel für ein Gedankenexperiment mit Brezn ist:<|end|>
<|endoftext|>


In [9]:
# Erzeugung des Outputs
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

 Ein Gedankenexperiment mit Brot könnte lauten: "Stellen Sie sich vor, Sie haben eine Brotscheibe, die aus einem unbekannten, neuen Kulturen hergestellten Getreide besteht. Sie möchten herausfinden, ob diese Brotart gesünder ist als die herkömmliche Weißbrotart. Sie entscheiden sich, eine Studie durchzuführen, in der Sie zwei Gruppen von Menschen haben. Eine Gruppe isst täglich die neue Brotart, während die andere Gruppe weiterhin Weißbrot isst. Nach einem Monat messen Sie die Blutzuckerspiegel und den Cholesterinspiegel beider Gruppen. Wenn die Gruppe, die die neue Brotart isst, niedrigere Werte aufweist, könnte dies darauf hindeuten, dass die neue Brotart gesünder ist."


## Aufgabe 2: Chain Prompting
Erzeugen Sie mit Hilfe mehrere Prompts einen Namen und einen Slogan für einen Brezn Verkaufsautomaten.
Nutzen Sie anschließend das Ergebnis um hierfür eine Produktbeschreibung in drei Sätzen zu generieren.


In [23]:
product_prompt = [
{"role": "user", "content": "Erzeuge einen Namen und einen Slogan für einen Brezen Verkaufsautomaten"}
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

 Name: "Brez'nator"

Slogan: "Brez'nator – Die Brez'n-Welt, wo jeder Brezen ein Geschenk ist!"


In [14]:
sales_prompt = [
{"role": "user", "content": f"Generiere eine Produktbeschreibung in 3 Sätzen zu folgendem Produkt: '{product_description}'"}
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

 Der Brez'nator ist nicht nur ein Snack, sondern eine Erfahrung, die jeden Tag neu enthüllt. Mit seinen vielfältigen Geschmacksrichtungen und der innovativen Brez'n-Welt-Konzeption bietet er eine Vielfalt an Geschenken für jeden Morgen. Er ist der perfekte Begleiter für jede Aktivität, sei es ein entspannter Morgen oder ein dynamischer Tag.


## Aufgabe3: Zero-shot Chain-of-Thought
Folgende Angabe ist gegeben:

Ein Metzger verkauft Weißwürste. Er hat 3 Pfund Weißwürste, die pro Pfund 4,50 € kosten und 240g Butter pro Pfund enthalten. Pro Weißwurst sind 12g Schweinefleisch verarbeitet. 1 Pfund (ca. 454g) ist ein 10-teiliges Paket. Eine Weißwurst wiegt ca. 50g. Ein Kunde möchte 10 Weißwürste kaufen, aber er hat nur 15 € dabei und möchte noch ein 250g Päckchen Butter dazu kaufen, das 1,80 € kostet. 

Nutzen Sie das LLM um eine oder mehrere der folgenden Fragen zu lösen. Funktioniert die Lösung mit oder ohne aufzeichnung der Schritte besser?

1. Wie viele Weißwürste sind in den 3 Pfund, die der Metzger hat?
2. Wie viel Gramm Fleisch sind in einer Weißwurst enthalten?
3. Wie viel wiegen die 10 Weißwürste, die der Kunde kaufen möchte, und wie viel kostet das?
4. Kauft der Kunde genug Weißwürste für ein Pfund, und ist es genug Fleisch für eine Mahlzeit für ihn und seine Familie?
5. Wie viel Geld bekommt der Metzger, wenn der Kunde die Weißwürste und die Butter bezahlt?
6. Was ist die Gesamtzahl an benötigten Weißwürsten für einen Einkauf von 2 Paketen à 1 Pfund plus Butter für ein 500g Paket?

In [17]:
Zeroshot_prompt=[
    {
        "role": "user", 
        "content": "Ein Metzger verkauft Weißwürste. Er hat 3 Pfund Weißwürste, die pro Pfund 4,50 € kosten und 240g Butter pro Pfund enthalten. Pro Weißwurst sind 12g Schweinefleisch verarbeitet. 1 Pfund (ca. 454g) ist ein 10-teiliges Paket. Eine Weißwurst wiegt ca. 50g. Ein Kunde möchte 10 Weißwürste kaufen, aber er hat nur 15 € dabei und möchte noch ein 250g Päckchen Butter dazu kaufen, das 1,80 € kostet. Wie viele Weißwürste sind in den 3 Pfund, die der Metzger hat? Denke Schritt für Schritt."
    },
    {
        "role": "assistant", 
        "content": "1 Pfund sind 10 Weißwürste, 3 * 10 = 30. Also sind 3 Pfund gleich 30 Weißwürste"
    },
    {
        "role": "user", 
        "content": "Ein Metzger verkauft Weißwürste. Er hat 3 Pfund Weißwürste, die pro Pfund 4,50 € kosten und 240g Butter pro Pfund enthalten. Pro Weißwurst sind 12g Schweinefleisch verarbeitet. 1 Pfund (ca. 454g) ist ein 10-teiliges Paket. Eine Weißwurst wiegt ca. 50g. Ein Kunde möchte 10 Weißwürste kaufen, aber er hat nur 15 € dabei und möchte noch ein 250g Päckchen Butter dazu kaufen, das 1,80 € kostet. Wie viel Gramm Fleisch sind in einer Weißwurst enthalten? Denke Schritt für Schritt."
    },
]
outputs = pipe(Zeroshot_prompt)
Zeroshot_pitch = outputs[0]["generated_text"]
print(Zeroshot_pitch)

 1 Pfund sind 10 Weißwürste, 3 * 10 = 30. Also sind 3 Pfund gleich 30 Weißwürste. 1 Weißwurst enthält 12g Schweinefleisch. Also sind 30 Weißwürste 30 * 12g = 360g Fleisch.


## Aufgabe 4: Tree-of-Thought: 
Erstellen Sie einen Prompt der folgende Bedingungen erfüllt:
- 3 Experten diskutieren
- Jeder Experte denkt Schrittweise denken.
- Die Experten teilen und vergleichen ihre Ergebnisse bevor sie zum nächsten Schritt wechseln
- Sobald ein Experte falsch liegt meldet, dieser sein falsches Ergebnis und scheidet anschließend aus.
- Die Gruppe soll folgende Frage beantworten:
  Wenn Sie 26 Brezn besitzen und 20 Brezn für das Mittagessen verbrauchen und 6 Brezn neu kaufen,"
    " wie viele Brezn haben Sie dann
- Die Experten sollen Ihr die Ergebnisse diskutieren.


## Aufgabe 5: Output Verification
Generiere ein Brezn Rezept in JSON format.

In [27]:
product_prompt = [
{"role": "user", "content": "Erzeuge ein Brot Rezept im JSON Format"},
{"role": "assistant", "content": "[{'name':'Brot'}, {'Zutaten': 'Mehl', 'Wasser'}], {'Zubereitung':'Alles verrühren, dann backen bis es braun ist'}"},
{"role": "user", "content": "Erzeuge ein Brezen Rezept im JSON Format"},
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

 [{'name':'Brezen'}, {'Zutaten': 'Weizenmehl, Salz, Hefe, Wasser, Olivenöl'}, {'Zubereitung':'Mehl mit Salz vermischen, Hefe mit Wasser auftragen, alles zu einem Teig verarbeiten, den Teig auf einem Teiglappen legen, mit Olivenöl bestreichen, mit einem Brezelstylzirkel ausrollen, die Brezen mit einem Brezelstylzirkel ausschneiden, die Brezen auf einem Backblech auslegen, 20 Minuten bei 220°C backen'}, {'Garantie':'Genießerisches, knuspriges Brot'}
